# Timeseries prediction of one pollutant at one station for one hour in the future

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys, os, pickle
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import plotting, modelling

%matplotlib inline

In [ ]:
# Load data and filter for a single station

df = pd.read_pickle('../data/processed/SO2_0.pkl')
df = df.loc[df['station'] == 101]


In [ ]:
# Create data for model

df_data = df['value']
df_data.index = df['datetime']
df_data.plot(subplots=True)


In [ ]:
# drop datetime

df_data = df_data.values


In [ ]:
# Standardize

train_split = 24*365*2

df_train_mean = df_data[:train_split].mean()
df_train_std = df_data[:train_split].std()

df_data = (df_data-df_train_mean)/df_train_std


In [ ]:
# split timeseries into train and validation

history_size = 20
future_size = 0

x_train, y_train = modelling.chop_data(df_data, 0, train_split,
                                       history_size,
                                       future_size)
x_val, y_val = modelling.chop_data(df_data, train_split, None,
                                   history_size,
                                   future_size)

## Begin with model definition in tensorflow

In [ ]:
# create train and validation objects using tensorflow Dataset

BATCH_SIZE = 256
BUFFER_SIZE = 10000

train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val = val.batch(BATCH_SIZE).repeat()

### Model Definition

In [ ]:
# define sequential tf.keras model

units = 1

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units, input_shape=x_train.shape[-2:]),
    tf.keras.layers.Dense(1)
])

In [ ]:
OPT = 'adam'
LOSS = 'mae'

model.compile(optimizer=OPT, loss=LOSS, metrics=['mae'])


In [ ]:
# check the output of the model is the correct shape

for x, y in val.take(1):
    print(model.predict(x).shape)
    

In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 2

model.fit(train, epochs=EPOCHS,
          steps_per_epoch=EVALUATION_INTERVAL,
          validation_data=val, validation_steps=50)

In [ ]:
model.summary()

In [ ]:
# save model and history

model_name = 'LSTM{}'.format(units)
subd = 'SO2/'
model_path = '../src/models/{}'.format(subd)

try:
    os.makedirs(model_path)
except FileExistsError:
    pass

model.save(model_path+'{}.h5'.format(model_name), save_format='tf')

modelling.save_history(model.history.history, model_name, subd)


### Plot loss history

In [ ]:
plotting.plot_train_history(model.history.history,
                            'Single Step Training and validation loss')

In [ ]:
# visualizing some predictions vs actuals

for x, y in val.take(3):
    plot = plotting.y_vs_yhat([x[0].numpy(),
                      y[0].numpy(),
                      model.predict(x)[0]],
                      0,
                      'Simple LSTM model')
    plot.show()